In [4]:
print('------------Training LgbmBoost------------')
%run ./Preprocessing.ipynb
import optuna

------------Training LgbmBoost------------
Preprocessed : xtrain, xtest, ytrain, ytest
(74736, 13) (18684, 13) (74736,) (18684,)


In [5]:
import lightgbm as lgb

from lightgbm import (LGBMClassifier, Dataset, cv, early_stopping)

from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score

In [6]:
static_param = {
    'objective': 'binary',
    'metric': 'auc',
    "verbose": -1,
}

opt_grid = {
    'early_stopping': 100,
    'number_of_trials': 100,
    'shuffle': True,
    'folds' : 10, 
    'cv': KFold(n_splits=10, random_state=None, shuffle=True),
    'time_constraint': 60 * 1
}


NameError: name 'KFold' is not defined

In [ ]:
def objective(trial):
        '''
    Objective function to optimise the hyperparamers of LGBM Classification 
    Args:
         trial object (object): Trials evaluating an objective function. 
         This object provides interfaces to get parameter suggestion, manage the trial’s state, and set/get user-defined attributes of the trial.

    Returns:
          Study  (object): Trained Study Object
    '''  
        
        param_grid = {
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.8),
            "max_depth": trial.suggest_int("max_depth", 2, 30),
            'min_child_weight': trial.suggest_float("min_child_weight", 1, 500),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 1),
            "colsample_bylevel": trial.suggest_float("colsample_bytree", 0.1, 1),
            "subsample": trial.suggest_float("subsample", 0.1, 1),
            "reg_alpha": trial.suggest_float("reg_alpha", 0, 5),
            "reg_lambda": trial.suggest_float("reg_lambda", 0, 5),
            'max_bin': trial.suggest_int('max_bin', 50, 100)
        }

        pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "auc")
        early_callback = lgb.early_stopping(5)

        full_grid = {**param_grid, **static_param}

        train_set = lgb.Dataset(xtrain, label=ytrain)

        cv_results = lgb.cv(full_grid,
                            train_set,
                            folds=opt_grid['cv'],
                            nfold=opt_grid['folds'],
                            shuffle=False,
                            callbacks=[pruning_callback, early_callback]
                            )

        scores = np.array(cv_results['%s-mean' % full_grid['metric']])

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        return np.mean(scores)

optuna.logging.set_verbosity(optuna.logging.CRITICAL)
study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
study.optimize(objective, n_trials=opt_grid['number_of_trials'], show_progress_bar=False, timeout=opt_grid['time_constraint'])
model_parameters = study.best_params

In [7]:
lgbm_model = LGBMClassifier()

In [8]:
#Set the parameters to the tuned parameters from the optuna study
lgbm_model.set_params(**model_parameters)

NameError: name 'model_parameters' is not defined

In [9]:
#Fit the model on the training data
model_lgbm=lgbm_model.fit(xtrain,ytrain)

#Make predictions. 
pred_lgbm=model_lgbm.predict(xtest)
probs_lgbm = model_lgbm.predict_proba(xtest)[:, 1]

pred_lgbm_train = model_lgbm.predict(xtrain)
probs_lgbm_train =  model_lgbm.predict_proba(xtrain)[:, 1]

print('-----------model_lgbm, pred_lgbm, probs_lgbm loaded-----------')

-----------model_lgbm, pred_lgbm, probs_lgbm loaded-----------
